In [ ]:
import craft_text_detector as craft

#### set image path and export folder directory

In [ ]:
image_name = 'idcard.png'
image_path = 'figures/' + image_name
output_dir = 'outputs/'

#### read image

In [ ]:
image = craft.read_image(image_path)

#### load models

In [ ]:
craft_net = craft.craft_detector(image=image,  cuda=True)

#### perform prediction

In [ ]:
text_threshold = 0.9
link_threshold = 0.2
low_text = 0.2
cuda = True  # False
show_time = False
# perform prediction
prediction_result = craft_net(image=image,
                         text_threshold=0.7,
                         link_threshold=0.4,
                         low_text=0.4,
                         target_size=1280,
                         show_time=True)


#### export detected text regions

In [ ]:
exported_file_paths = craft.export_detected_regions(
    image_path=image_path,
    image=image,
    regions=prediction_result["boxes"],
    output_dir=output_dir,
    rectify=True
)

#### export heatmap, detection points, box visualization

In [ ]:
craft.export_extra_results(
    image_path=image_path,
    image=image,
    regions=prediction_result["boxes"],
    heatmaps=prediction_result["heatmaps"],
    output_dir=output_dir
)
